In [1]:
# importing libraries
from ipywidgets import *
from IPython.display import clear_output, display, Javascript
from tkinter import Tk, filedialog
import pandas as pd
import ipywidgets as widgets
import plotly.express as px
import random
import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, hamming_loss, classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier
from skmultilearn.adapt import BRkNNaClassifier
from sklearn.model_selection import train_test_split
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import MLkNN
import seaborn as sns
import numpy as np

In [2]:
def loginLogout():
    return True

In [2]:
out11 = widgets.Output()

def on_select_Column_Change(event):
    if (columnNullList.value !=0):
        dic = getNullDict()
        key_list = list(dic.keys())
        value_list = list(dic.values())
        position = value_list.index(columnNullList.value)
        s = key_list[position]
        getNullValuesfor(getGeneralDataset(),s)
    else:
        out11.clear_output()

columnNullList = Dropdown(
            options={'',0},
            value=0,
            description='Choose attribute:',
            style= {'description_width':'auto'}
            )
columnNullList.observe(on_select_Column_Change, names=['value'])

In [3]:
# Functions

# 1 - Showing file chooser dialog -> Returns file selected
def select_files(b):
    #clear_output()                                        # Button is deleted after it is clicked.
    root = Tk()
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    b.files = filedialog.askopenfilename(multiple=False)   # List of selected files will be set button's file attribute.
    return b.files                                         # Print the list of files selected.
# -------------------------------------------------------------------------------------------------------------------------

# 2 - Read dataset -> This function is so specific for "PersonTasks" dataset -> Returns dataset
def read_file(file_path):
    # Reading dataset:
    if withConsole==True:
        outputText.append('>> Reading dataset file : '+str(file_path))
    dataset = pd.read_json(file_path,orient="columns");
    if checkIfPersonTasksDataset(dataset) == True:
        # Flattening dataset
        if withConsole==True:
            outputText.append('>> Flattening dataset file : '+str(file_path))
        dataset = showJSONDataset(file_path)
        # Check how many people in the dataset
        numberOfPersons = dataset.personID.unique();
        if withConsole==True:
            outputText.append('>> Found '+str(len(numberOfPersons))+" volunteer(s) in dataset")
        # Takes the first person
        selectedPersonID = numberOfPersons[0]
        if withConsole==True:
            if (len(numberOfPersons)>1):
                outputText.append('>> Filtering dataset to one volunteer which personID = '+str(selectedPersonID))
        # Filter dataset
        dataset = dataset[dataset.personID==selectedPersonID]
        # Replacing personID by 1 and adding personName with a random number from namesList list
        # dataset = dataset.drop(["personID"],axis=1)
        dataset.personID = dataset.personID.replace(selectedPersonID,1)
        outputText.append('>> Changing personID from '+str(selectedPersonID)+' to 1')
        namesList = ['Jack','Jordan','Axel','Ian','Luis','Nicolas',
            'Sophia','Charlotte','Scarlett','Ivy','Lucy','Ariana']
        name = random.choice(namesList)
        dataset['personName'] = name
        if withConsole==True:
            outputText.append('>> Adding personName attribute with name : '+str(name))
        # Visualizing personID, taskType1, taskType2 and taskDuration
        if withConsole==True:
            outputText.append('>> Making a Sunburst Visualization for taskType1, taskType2 and taskDuration for : '+str(name))
        datasetVisualization = dataset[['personName','taskType1','taskType2','taskDuration']]
        fig = px.sunburst(datasetVisualization, path=[datasetVisualization["personName"].tolist(),
                                                      datasetVisualization["taskType1"].tolist(), 
                                                      datasetVisualization["taskType2"].tolist()], 
                          values=datasetVisualization["taskDuration"].tolist())
        fig.update_layout(showlegend=True,
                          title_text="Sunburst chart for: "+ name)
        # Statistics for all attributes
        if withConsole==True:
            outputText.append('>> Making some statistics for: '+str(name))
        datasetNumericStatistics = dataset.describe()
        datasetStatistics = dataset.describe(include='all')
        
        # Getting all Null/NaN columns which null/nan values != 0 and show them in columnNullList drop down list
        allNaNNullValues = showNullNaNValues(dataset)
        dropDownValue = 1;
        allNaNNullValuesNotZero = allNaNNullValues[allNaNNullValues["# of null/nan values"]!=0].reset_index(drop=True)
        global nullDict
        nullDict = {'':0}
        for i in range(len(allNaNNullValuesNotZero)):
            nullDict[str(allNaNNullValuesNotZero["Columns"][i])] = dropDownValue;
            dropDownValue = dropDownValue + 1
        columnNullList.options = nullDict
        columnNullList.value = 0
        
        with output:
            out1 = widgets.Output()
            out2 = widgets.Output()
            out3 = widgets.Output()
            out4 = widgets.Output()
            out5 = widgets.Output()
            out7 = widgets.Output()
            out10 = widgets.Output()
            if withConsole==True:
                out6 = widgets.Output()
                finalResult = Accordion(children=[out6,out1,out2,out3], selected_index=None)
                finalResult.set_title(0,"Console:")
                finalResult.set_title(1,"Dataset View:")
                finalResult.set_title(2,"Visulaization:")
                finalResult.set_title(3,"Statistics:")
                # Tab inside Statistics accordion
                tabChildren = [out4,out5,out7,out10]
                tab = Tab(children=tabChildren)
                tab.set_title(0,'Numeric data:')
                tab.set_title(1,'All data:')
                tab.set_title(2,'Null/NaN values:')
                tab.set_title(3,'Null/NaN rows:')
                
                consoleText = Textarea(value="Console:", rows=10, disabled=True, layout=Layout(width="auto"))
                for text in outputText:
                    consoleText.value = consoleText.value + "\r" + text
            else:
                finalResult = Accordion(children=[out1,out2,out3], selected_index=None)
                finalResult.set_title(0,"Dataset View:")
                finalResult.set_title(1,"Visulaization:")
                finalResult.set_title(2,"Statistics:")
                # Tab inside Statistics accordion
                tabChildren = [out4,out5,out7,out10]
                tab = widgets.Tab(children=tabChildren)
                tab.set_title(0,'Numeric data:')
                tab.set_title(1,'All data:')
                tab.set_title(2,'Null/NaN values:')
                tab.set_title(3,'Null/NaN rows:')
                
            display(finalResult)
            with out1:
                display(dataset)
            with out2:
                display(fig)
            with out3:
                display(tab)
            with out4:
                display(datasetNumericStatistics)
            with out5:
                display(datasetStatistics)
            with out7:
                display(allNaNNullValues)
            with out10:
                # out11 is global
                display(VBox([columnNullList,out11]))
            if withConsole==True:
                with out6:
                    display(consoleText)
            '''
            display(title1)
            display(dataset)
            display(title2)
            display(fig)
            display(title3)
            display(datasetStatistics)
            '''
        enableDisableButton(preprocessingButton,1)
        return dataset
    else:
        errorMessage = getMessage('error','You select a wrong dataset! Please select a suitable one!')
        enableDisableButton(preprocessingButton,0)
        with output:
            display(errorMessage)
        return False

def getNullDict():
    return nullDict
    
def getNullValuesfor(dataset,columnName):
    out11.clear_output()
    nullValues = dataset[dataset[columnName].isna()]
    with out11:
        display(nullValues)    
# -------------------------------------------------------------------------------------------------------------------------
# 3 - After reading a dataset -> ordering events
def order_events(b):
    fileselect1.icon = "spinner"
    global outputText;
    outputText = [];
    # Checking if we want it with console or not
    global withConsole
    withConsole = False
    file = select_files(b)
    if '.json' in file:
        output.clear_output()
        # defining global variable in order accessing it in the next cells.
        global finalDataset
        finalDataset = read_file(file)
        fileselect1.icon = "file"
        return finalDataset
    else:
        output.clear_output()
        fileselect1.icon = "file"
        with output:
            informationMessage = getMessage('information','You have to choose a dataset with a JSON format!!')
            enableDisableButton(preprocessingButton,0)
            display(informationMessage)

# ------------------------------------------------------------------------------------------------------------------------            
def order_eventsWithOutput(b):
    fileselect2.icon = "spinner"
    global outputText;
    outputText = [];
    # Checking if we want it with console or not
    global withConsole
    withConsole = True
    file = select_files(b)
    if '.json' in file:
        output.clear_output()
        # defining global variable in order accessing it in the next cells.
        global finalDataset
        finalDataset = read_file(file)
        fileselect2.icon = "info-circle"
        return finalDataset
    else:
        fileselect2.icon = "info-circle"
        output.clear_output()
        with output:
            informationMessage = getMessage('information','You have to choose a dataset with a JSON format!!')
            enableDisableButton(preprocessingButton,0)
            display(informationMessage)

# ------------------------------------------------------------------------------------------------------------------------
# 4 - Check if this is the target dataset "PersonTasks" dataset
def checkIfPersonTasksDataset(dataset):
    columns = dataset.columns
    if len(columns) == 13:
        if ('taskType1' in columns) & ('taskType2' in columns) & ('taskDuration' in columns):
            return True
        else:
            return False
    else:
        return False
    
# ------------------------------------------------------------------------------------------------------------------------
# 5 - Messages
def getMessage(messsageType,messageText):
    finalMessage = ''
    if messsageType == 'error':
        finalMessage = Button(description=str(messageText),icon="remove",layout=Layout(width='auto'),button_style="danger",disabled=True)
        # widgets.HTML(value='<img style="float: left" src="./icons/error.png" width="30px" height="30px"/>&nbsp<h4 style="color:red;">Error: <h4>&nbsp <span>'+  +'</span>')
    else:
        finalMessage = Button(description=str(messageText),icon="info-circle",layout=Layout(width='auto'),button_style="info",disabled=True)
        # widgets.HTML(value='<img style="float: left" src="./icons/information.png" width="30px" height="30px"/>&nbsp<h4 style="color:blue;">Information: <h4>&nbsp <span>'+ str(messageText) +'</span>')
    return finalMessage

# ------------------------------------------------------------------------------------------------------------------------
# 6 - These three functions to flatten dataset if it has nested JSON code:
# Knowing nested JSON columns in dataset
def extractNestedColumns(dataframe):
    nestedColumns = [];
    col = dataframe.columns;
    for column in col:
        value = dataframe[column][0];
        if "{" in str(value):
              nestedColumns.append(column);
    return nestedColumns

def splittingNestedColumns(dataframe):
    # Taking nested json columns in dataframe
    nestedColumns = extractNestedColumns(dataframe);
    if len(nestedColumns)!=0:
        for i in nestedColumns:
            # taking every column in nestedColumns attributes and adding it to the original dataframe
            p = pd.DataFrame(dataframe[i]);
            f = [];
            for j in p[i]:
                f.append(j);
            s = pd.DataFrame(f);
            scol = s.columns;
            for k in scol:
                s = s.rename(columns={k:"{}{}{}".format(i,"_",k)})
            scol = s.columns;
            for n in scol:
                dataframe[n] = s[n];
            for j in scol:
                dataframe[j] = s[j];
            newDataFrame = dataframe.drop(nestedColumns,axis=1) 
    return newDataFrame

# A function to show [flattened] dataset
def showJSONDataset(datasetPath):
    # Reading the JSON File and put it in a variable
    datasetVariable = pd.read_json(datasetPath,orient='columns')
    datasetVariable = splittingNestedColumns(datasetVariable)
    return datasetVariable

# ------------------------------------------------------------------------------------------------------------------------
# 7 - Function to enable/disable a button 0-> disable , 1->enable
def enableDisableButton(buttonName,value):
    if value == 0:
        buttonName.layout = Layout(width="0",height="0")
        output2.clear_output()
        buttonName.disabled=False
        '''
        buttonName.button_style="danger"
        buttonName.disabled=True
        '''
    else:
        buttonName.layout = Layout(width="auto",height="auto")
        output2.clear_output()
        buttonName.disabled=False
        '''
        buttonName.button_style="success"
        buttonName.disabled=False
        '''

# -----------------------------------------------------------------------------------------------------------------------
# 8 - Function to show categorical/numeric attributes with null/nan values
def showNullNaNValues(dataset):
    dataset.convert_dtypes().dtypes;
    col = dataset.columns;
    nullValues = list(dataset.isnull().sum());
    colType = [];
    for i in col:
        if (dataset[i].dtype == "O") | (dataset[i].dtype == "bool"):
            colType.append("Categorical");
        else:
            colType.append("Numeric");

    # Making dataframe
    columnsDict={"Columns":list(col),"Types":list(colType),"# of null/nan values":nullValues};
    df = pd.DataFrame(columnsDict);
    return df

# -----------------------------------------------------------------------------------------------------------------------
# 9 - Function to get general dataset
def getGeneralDataset():
    return finalDataset.copy();

# <font color="blue"><center>Learning Volunteer Competencies</center></font>
***
## <img style="float: left" src="./icons/mode.png" width="25px" height="25px"/>&nbsp; Assessment Mode:

In [4]:
assessmentMode = Dropdown(
            options={'Learning Method': 1},
            value=1,
            description='Assessment Mode : ',
            style= {'description_width':'auto'}
            )

display(assessmentMode)

Dropdown(description='Assessment Mode : ', options={'Learning Method': 1}, style=DescriptionStyle(description_…

***

## <img style="float: left" src="./icons/dataset.png" width="25px" height="25px"/>&nbsp; Dataset:

In [5]:
# For printing widgets
output = widgets.Output()

# Buttons
layout = widgets.Layout(width='auto')
fileselect1 = Button(description="Select Dataset", icon="file", layout=layout, tooltip="Select dataset and show the result")
fileselect1.on_click(order_events)

fileselect2 = Button(description="Select Dataset with console",
                     icon="info-circle", layout=layout, tooltip="Select dataset and show the result step by step")
fileselect2.on_click(order_eventsWithOutput)

buttons = HBox([fileselect1,fileselect2])


display(buttons)
display(output)

Output()

***
## <img style="float: left" src="./icons/pre-processing.png" width="25px" height="25px"/>&nbsp; Pre-processing:

In [26]:
# functions for preprocessing
def extractingDateTimeFeatures(dataset,column1,column2):
    dataset[column1] = pd.to_datetime(dataset[column1])
    dataset[column2] = pd.to_datetime(dataset[column2])

    # Extracting date and time
    dateFrom = [d.date() for d in dataset[column1]]
    dateTo = [d.date() for d in dataset[column2]]
    timeFrom = [d.time() for d in dataset[column1]]
    timeTo = [d.time() for d in dataset[column2]]

    # Insert them in dataset in specific locations
    dataset.insert(7,"dateFrom",dateFrom)
    dataset.insert(8,"dateTo",dateTo)
    dataset.insert(9,"timeFrom",timeFrom)
    dataset.insert(10,"timeTo",timeTo)
    
    # Return dataset
    return dataset

def deleteAttributes(dataset,listOfAttributes):
    dataset = dataset.drop(listOfAttributes,axis=1)
    return dataset

def getTaskDuration(t1,t2):
    duration = t2-t1
    duration_in_s = duration.total_seconds()
    duration_in_h = divmod(duration_in_s, 3600)
    duration_in_m = round(duration_in_h[1] / 3600,2)
    finalDuration = duration_in_h[0] + duration_in_m
    return finalDuration

# This function is specific for PersonTasks dataset
def handlingTaskDuration(dataset):
    numberOfRows = len(dataset);
    numberOfAffectedRows = 0;
    for i in range(numberOfRows):
        taskD = dataset.taskDuration[i];
        if (pd.isna(taskD)):
            t1 = pd.to_datetime(dataset.taskDateFrom[i]);
            t2 = pd.to_datetime(dataset.taskDateTo[i]);
            taskDuration = getTaskDuration(t1,t2)
            dataset.loc[i, "taskDuration"] = taskDuration;
            numberOfAffectedRows = numberOfAffectedRows + 1;
    return dataset, numberOfAffectedRows

# A function to return number of tasks that are accomplished between two specific hours
# Parameters hour1, hour2 from string datatype
def getNumberOfTasksBetweenTwoHours(dataset,column,personID,hour1,hour2):
    resultedRows = dataset[dataset.personID==personID];
    
    list1=[]; # A list of all hours between the two defined hours
    hoursRange = pd.date_range(hour1,hour2,freq="1min")
    for data in hoursRange:
        list1.append(data.time())

    filteredColumn = resultedRows[column];
    list2 = []; # list of all values in column which its value is in list1
    for i in filteredColumn:
        if (i in list1):
            list2.append(i)
            
    return len(list2)

def getGeneratedDataset(dataset):
    # Column of generated dataset
    personName = [];
    NumberOfAccomplishedTasks = [];
    OrganizationManagementProportion = [];
    TrainingLearningProportion = [];
    CoorporationProportion = [];
    VarietyWorkProportion = [];
    StartWorkingMorning = [];
    WorkingLongTime = [];
    taskLocation = [];
    
    numberOfVolunteers = list(dataset.personID.unique());
    
    for pId in numberOfVolunteers:
        # Taking tasks for each volunteer
        volunteerTasks = dataset[dataset.personID==pId]

        # Begin extracting new attributes
        # 1 - personName
        volunteerName = volunteerTasks.personName.unique()
        personName.append(volunteerName[0]);

        # 2- Knowing the number of tasks:
        totalNumberOfTasks = len(volunteerTasks.taskType1);
        NumberOfAccomplishedTasks.append(totalNumberOfTasks)

        # 3 - OrganizationManagementProportion -> Number of tasks that from
        # taskType1='Organisation | Verwaltung', 'Bewerb | Leistungsprüfung (inkl. Vorbereitung)' and 'Einsatz'
        # Number between 0-1.
        OrganizationManagementProportion.append((round(len(volunteerTasks[((volunteerTasks.taskType1=='Organisation | Verwaltung') |
                                                         (volunteerTasks.taskType1=='Einsatz'))]) / totalNumberOfTasks,2)))

        # 4 - TrainingLearningProportion -> Number of tasks that from 
        # taskType1='Schulung', 'Übung' and 'Kurs | Weiterbildung'
        # Number between 0-1.
        TrainingLearningProportion.append((round(len(volunteerTasks[((volunteerTasks.taskType1=='Schulung') |
                                                         (volunteerTasks.taskType1=='Übung') |
                                                         (volunteerTasks.taskType1=='Kurs | Weiterbildung'))]) / totalNumberOfTasks,2)))

        # 5 - CoorporationProportion -> Number of tasks that from
        # taskType1='Jugend'
        # Number between 0-1.
        CoorporationProportion.append(round(len(volunteerTasks[(volunteerTasks.taskType1=='Jugend')])/ totalNumberOfTasks,2))

        # 6 - VarietyWorkProportion -> Number of tasks that from 
        # taskType1='Sonstiges'
        # Number between 0-1.
        VarietyWorkProportion.append(round(len(volunteerTasks[(volunteerTasks.taskType1=='Sonstiges')|
                                                         (volunteerTasks.taskType1=='Bewerb | Leistungsprüfung (inkl. Vorbereitung)') |
                                                             (volunteerTasks.taskType1=='Arbeiten in der FW')])/ totalNumberOfTasks,2))

        # 7 - StartWorkingMorning -> Number of tasks that from 
        # timeFrom and using getNumberOfTasksBetweenTwoHours function "08:00:00" and "12:00:00"
        # Number between 0-1.
        StartWorkingMorning.append(round(getNumberOfTasksBetweenTwoHours(volunteerTasks,'timeFrom',pId,'08:00:00','12:00:00')/totalNumberOfTasks,2))

        # 8 - WorkingLongTime -> Number of tasks in which 
        # Taking the mean of all volunteer's taskDuration and compare each task to it and take all them that >= mean 
        # Number between 0-1 and Expresses about telorate work prassure.
        taskDurationMean = volunteerTasks.taskDuration.mean();
        WorkingLongTime.append(round(len(volunteerTasks[volunteerTasks.taskDuration >= taskDurationMean])/totalNumberOfTasks,2))

        # 9 - taskLocation -> Number of unique locations
        # Number between 0-1.
        # -> Mobility.
        taskLocation.append(round(len(volunteerTasks.taskLocation.unique())/totalNumberOfTasks,2))

    # ---------------------------------------------------------------------------------------------------------
    # -------------------------------------------GETTING DICTIONARY--------------------------------------------
    # ---------------------------------------------------------------------------------------------------------
    generatedDatasetDict = {
        'personName':personName,
        'NumberOfAccomplishedTasks':NumberOfAccomplishedTasks,
        'OrganizationManagementProportion':OrganizationManagementProportion,
        'TrainingLearningProportion':TrainingLearningProportion,
        'CoorporationProportion':CoorporationProportion,
        'VarietyWorkProportion': VarietyWorkProportion,
        'StartWorkingMorning': StartWorkingMorning,
        'WorkingLongTime': WorkingLongTime,
        'taskLocation':taskLocation
    }

    # ---------------------------------------------------------------------------------------------------------
    # -------------------------------------------GETTING DATAFRAME---------------------------------------------
    # ---------------------------------------------------------------------------------------------------------
    generatedDatasetDataFrame = pd.DataFrame(generatedDatasetDict)
    return generatedDatasetDataFrame

def getFinalPreprocessedDataset():
    return finalPreprocessedDataset.copy()

# ------------------------------------------------------------------------------------------------------------------------------
# activating prediction button
def activatePredictionButton():
    if (runModelButton.disabled==False) & (preprocessingButton.disabled==True):
        predictionButton.disabled=False
    else:
        predictionButton.disabled=True

def preprocessingOnClickEvent(event):
    preprocessedDataset = getGeneralDataset()
    preprocessingOutput = [];
    preprocessingOutput.append(">> Extracting time and date from taskDateFrom and taskDateTo attributes.")
    preprocessedDataset = extractingDateTimeFeatures(preprocessedDataset,'taskDateFrom','taskDateTo')
    listOfDeletedAttributes = ['taskName', 'iVolunteerSource', 'iVolunteerUUID', 'taskGeoInformation_latitude', 'taskGeoInformation_longitude','taskGeoInformation_gridID']
    preprocessedDataset = deleteAttributes(preprocessedDataset,listOfDeletedAttributes)
    preprocessingOutput.append(">> Deleting unnecessary "+str(len(listOfDeletedAttributes))+" attributes:")
    for attribute in listOfDeletedAttributes:
        preprocessingOutput.append(">> Deleting "+str(attribute)+".")
    nullValuesList = showNullNaNValues(preprocessedDataset)
    nullValuesList = nullValuesList[nullValuesList["# of null/nan values"] != 0].reset_index(drop=True)
    for i in range(len(nullValuesList)):
        if (nullValuesList["# of null/nan values"][i] > 1):
            preprocessingOutput.append('>> Found '+str(nullValuesList["# of null/nan values"][i])+' Null/NaN values in '+str(nullValuesList['Columns'][i])+' attribute')
        else:
            preprocessingOutput.append('>> Found '+str(nullValuesList["# of null/nan values"][i])+' Null/NaN value in '+str(nullValuesList['Columns'][i])+' attribute')
    preprocessingOutput.append('>> Handling Null/NaN values.')
    preprocessedDataset, numberOfAffectedRows = handlingTaskDuration(preprocessedDataset)
    preprocessingOutput.append('>> '+str(numberOfAffectedRows)+' rows has affected and handled!!')
    preprocessingOutput.append('>> Making dataset fit the learning model......')
    global finalPreprocessedDataset
    finalPreprocessedDataset = getGeneratedDataset(preprocessedDataset)
    with output2:
        out9 = widgets.Output()
        out12 = widgets.Output()
        preporcessingResult = Accordion(children=[out9,out12], selected_index=None)
        preporcessingResult.set_title(0,"Console for pre-processing step:")
        preporcessingResult.set_title(1,"Pre-processed Dataset:")
        preprocessingConsoleText = Textarea(value="Preprocessing Console:", rows=10, disabled=True, layout=Layout(width="auto"))
        for text in preprocessingOutput:
            preprocessingConsoleText.value = preprocessingConsoleText.value + "\r" + text
        display(preporcessingResult)
        with out9:
            display(preprocessingConsoleText)
        with out12:
            display(finalPreprocessedDataset)
    preprocessingButton.disabled = True
    activatePredictionButton()

In [22]:
output2 = widgets.Output()
preprocessingButton = Button(description="Make preprocessing", layout=Layout(width="0", height="0"), icon='gears')
preprocessingButton.on_click(preprocessingOnClickEvent)

display(preprocessingButton)
display(output2)

Button(description='Make preprocessing', icon='gears', layout=Layout(height='0', width='0'), style=ButtonStyle…

Output()

***
## <img style="float: left" src="./icons/learningModel.png" width="25px" height="25px"/>&nbsp; Learning Model:

In [40]:
# Functions for learning model

# When clicking EACH Classifier button
def check_event(clickEvent):
    tempButton = clickEvent
    if (tempButton.icon == 'plus-square'):
        if tempButton.description not in choosenClassifier:
            choosenClassifier.append(tempButton.description)
        tempButton.icon = 'minus-square'
        tempButton.button_style='primary'
    else:
        tempButton.icon = 'plus-square'
        for i in choosenClassifier:
            if i == tempButton.description:
                choosenClassifier.remove(i)
                tempButton.button_style=''
    checkForStepper()
    checkIfOneSelected(listOfButtons)

def checkForStepper():
    if (MLkNNButton.button_style == '') & (BRMLkNNButton.button_style == ''):
        k.disabled = True
    else:
        k.disabled = False
# ------------------------------------------------------------------------------------------------------------------------------
# When selecting "select all unselected" and "unselect all selected" buttons
def selectAllButtonOnClick(eventClick):
    tempButton = eventClick
    if tempButton.description=='Select All Unselected':
        for button in listOfButtons:
            if button.button_style == '':
                button.click()
    else:
        for button in listOfButtons:
            if button.button_style == 'primary':
                button.click()
    checkIfOneSelected(listOfButtons)

# ------------------------------------------------------------------------------------------------------------------------------
# Activating "Run Model" button
def checkIfOneSelected(listOfButtons):
    nb = 0
    for button in listOfButtons:
        if button.button_style=='primary':
            nb = nb + 1
    if nb>=1:
        runModelButton.disabled=False
    else:
        runModelButton.disabled=True
        
# ------------------------------------------------------------------------------------------------------------------------------
# Filling dropListClassifier drop down list according to choosen classifiers
def fillDropListClassifier(dropList,choosenClassifier):
    try:
        dropList.options = {}
    except:
        dropList.options = {}
    index = 0;
    classifierDict = {}
    for classifier in choosenClassifier:
        classifierDict[str(classifier)] = index
        index = index + 1
    dropList.options = classifierDict
# ------------------------------------------------------------------------------------------------------------------------------
# -----------------------------------------------LEARNING MODEL FUNCTIONS-------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------------
# For determining quality for each classifier in order to give an advice
def determineQuality(accuracyValue):
    if accuracyValue < 0.5:
        Quality.append('BAD')
    elif 0.5 <= accuracyValue <= 0.7:
        Quality.append('So-So')
    elif 0.71 <= accuracyValue <= 0.88:
        Quality.append('GOOD')
    else:
        Quality.append('VERY GOOD')

# ------------------------------------------------------------------------------------------------------------------------------
# PT_build_Model for Problem Transformation methods
def PT_build_Model(model,mlb_estimater,classifierName,algorithmName,Xtrain,ytrain,Xtest,ytest):
    Methods.append('Problem Transformation')
    clf = mlb_estimater(model,require_dense=[False,True])
    clf.fit(Xtrain,ytrain)
    # Prediction
    clf_predictions = clf.predict(Xtest)
    # Check the accuracy score and hamming loss
    acc = accuracy_score(ytest,clf_predictions)
    hmm = hamming_loss(ytest,clf_predictions)
    ClassifierName.append(classifierName)
    MethodsAlgorithms.append(algorithmName)
    accuracyArray.append(acc)
    hammingArray.append(hmm)
    determineQuality(acc)
    return clf

# ------------------------------------------------------------------------------------------------------------------------------
# AA_build_Model for Alogrithm Adaptation methods
def AA_build_Model(algorithm,kN,classifierName,algorithmName,X_train,y_train,X_test,y_test):
    Methods.append('Algorithm Adaptation')
    if algorithm==MLkNN:
        classifier = algorithm(k=kN,s=0.5)
    else:
        classifier = algorithm(k=kN)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    acc = accuracy_score(y_test,y_pred)
    hmm = hamming_loss(y_test,y_pred)
    ClassifierName.append(classifierName)
    MethodsAlgorithms.append(algorithmName)
    accuracyArray.append(acc)
    hammingArray.append(hmm)
    determineQuality(acc)
    return classifier 

# ------------------------------------------------------------------------------------------------------------------------------
# Reading training dataset -> Specific function
def readTrainingDataset():
    dataset = pd.read_json('./dataset/finalDataset1.json',orient='columns')
    return dataset

# ------------------------------------------------------------------------------------------------------------------------------
# Mapping classifiers
def MapClassifierToModelsWithFinalResult(choosenClassifier,numberOfKNN,X_train, X_test, y_train, y_test):
    listOfRunningClassifiers = []
    
    # For giving final result
    global Methods 
    Methods = []
    global ClassifierName 
    ClassifierName = []
    global MethodsAlgorithms
    MethodsAlgorithms = []
    global accuracyArray
    accuracyArray = []
    global hammingArray
    hammingArray = []
    global Quality
    Quality = []
    
    for classifier in choosenClassifier:
        if classifier == 'Binary Relevance with NomialNB':
            # BinaryRelevance
            BR1 = PT_build_Model(MultinomialNB(),BinaryRelevance,'BinaryRelevance','MultinominalNB',X_train,y_train,X_test,y_test)
            listOfRunningClassifiers.append(BR1)
        elif classifier == 'Binary Relevance with RandomForestClassifier':
            # BinaryRelevance
            BR2 = PT_build_Model(RandomForestClassifier(),BinaryRelevance,'BinaryRelevance','RandomForestClassifier',X_train,y_train,X_test,y_test)
            listOfRunningClassifiers.append(BR2)
        elif classifier == 'ClassifierChain with NomialNB':
            # ClassifierChain
            CC1 = PT_build_Model(MultinomialNB(),ClassifierChain,'ClassifierChain','MultinominalNB',X_train,y_train,X_test,y_test)
            listOfRunningClassifiers.append(CC1)
        elif classifier == 'Classifier Chain with RandomForestClassifier':
            # ClassifierChain
            CC2 = PT_build_Model(RandomForestClassifier(),ClassifierChain,'ClassifierChain','RandomForestClassifier',X_train,y_train,X_test,y_test)
            listOfRunningClassifiers.append(CC2)
        elif classifier == 'LabelPowerset with NomialNB':
            # LabelPowerset
            LP1 = PT_build_Model(MultinomialNB(),LabelPowerset,'LabelPowerset','MultinominalNB',X_train,y_train,X_test,y_test)
            listOfRunningClassifiers.append(LP1)
        elif classifier == 'LabelPowerset with RandomForestClassifier':
            # LabelPowerset
            LP2 = PT_build_Model(RandomForestClassifier(),LabelPowerset,'LabelPowerset','RandomForestClassifier',X_train,y_train,X_test,y_test)
            listOfRunningClassifiers.append(LP2)
        elif classifier == 'MLkNN':
            # ML-KNN
            MK1 = AA_build_Model(MLkNN,numberOfKNN,'MLkNN','MLkNN',X_train,y_train,X_test,y_test)
            listOfRunningClassifiers.append(MK1)
        elif classifier == 'BRMLkNN':
            # ML-KNN
            MK2 = AA_build_Model(BRkNNaClassifier,numberOfKNN,'BRkNNaClassifier','BRkNNaClassifier',X_train,y_train,X_test,y_test)
            listOfRunningClassifiers.append(MK2)

    # Making Final DataFrame
    dataDict = {'Method': Methods,
                'Classifier':ClassifierName,
                'Algorithm': MethodsAlgorithms,
                'Accuracy': accuracyArray,
                'Hamming Loss': hammingArray,
                'Quality': Quality}
    # Making a dataframe
    resultDf = pd.DataFrame(dataDict)
    
    # Returning final result and list of running classifiers
    return listOfRunningClassifiers, resultDf

# ------------------------------------------------------------------------------------------------------------------------------
def appendTextToConsole(textAreaPane,Text):
    textAreaPane.value = textAreaPane.value + "\r" + Text

# ------------------------------------------------------------------------------------------------------------------------------
# Train model
def runModelClickEvent(event):
    runModelButton.icon = "spinner"
    output3.clear_output()
    fillDropListClassifier(dropListClassifier,choosenClassifier)
    # For console
    out13 = widgets.Output()
    trainingResult = Accordion(children=[out13], selected_index=0)
    trainingConsoleText = Textarea(value="Training Console:", rows=10, disabled=True, layout=Layout(width="auto"))
    trainingResult.set_title(0,"Training Console:")
    
    with output3:
        display(trainingResult)
        with out13:
            display(trainingConsoleText)
    
    # Reading training dataset
    trainingDataset = readTrainingDataset()
    datasetRows = trainingDataset.shape[0]
    datasetColumns = trainingDataset.shape[1]
    appendTextToConsole(trainingConsoleText,"▶ Reading training dataset with size: "+str(datasetRows)+" rows * "+str(datasetColumns)+" columns")
    
    # Spliting training dataset
    X = trainingDataset[["NumberOfAccomplishedTasks", "OrganizationManagementProportion", "TrainingLearningProportion", "CoorporationProportion", "VarietyWorkProportion", "StartWorkingMorning", "WorkingLongTime", "taskLocation"]].values
    y = trainingDataset[["SelfManagement","WillingnessToLearn","Energy","Persistence","Mobility"]].values
    
    appendTextToConsole(trainingConsoleText,"▶ Droping personID column..")
    
    appendTextToConsole(trainingConsoleText,"▶ Spliting dataset into training set (80%) and test set (20%):")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
    trainingsetRows = X_train.shape[0]
    trainingsetColumns = X_train.shape[1]
    testsetRows = X_test.shape[0]
    testsetColumns = X_test.shape[1]
    appendTextToConsole(trainingConsoleText,"  ▶▶ Training set size: "+str(trainingsetRows)+" rows * "+str(trainingsetColumns)+" columns")
    appendTextToConsole(trainingConsoleText,"  ▶▶ Test set size: "+str(testsetRows)+" rows * "+str(testsetColumns)+" columns")
    
    global listOfTrainedRunningClassifiers
    
    numberOfKNN = k.value
    listOfTrainedRunningClassifiers, adviceResult = MapClassifierToModelsWithFinalResult(choosenClassifier,numberOfKNN,X_train, X_test, y_train, y_test)
    
    for item in choosenClassifier:
        appendTextToConsole(trainingConsoleText,"▶ Training model with "+str(item))
    
    
    adviceResultVeryGood = adviceResult[adviceResult.Quality=='VERY GOOD'].reset_index(drop=True)
    adviceResultGood = adviceResult[adviceResult.Quality=='GOOD'].reset_index(drop=True)
    adviceResultSoSo = adviceResult[adviceResult.Quality=='So-So'].reset_index(drop=True)
    adviceResultBAD = adviceResult[adviceResult.Quality=='BAD'].reset_index(drop=True)
    listOfPrettyGoodResult = []
    for vgr in range(len(adviceResultVeryGood)):
        text = str(adviceResultVeryGood.Classifier[vgr]) + ' with ' + str(adviceResultVeryGood.Algorithm[vgr])
        listOfPrettyGoodResult.append(Button(description=text,layout=Layout(width="auto"),disabled=True,icon='star', button_style='success'))
    for gr in range(len(adviceResultGood)):
        text = str(adviceResultGood.Classifier[gr]) + ' with ' + str(adviceResultGood.Algorithm[gr])
        listOfPrettyGoodResult.append(Button(description=text,layout=Layout(width="auto"),disabled=True,icon='star-half-o', button_style='primary'))
    for ssr in range(len(adviceResultSoSo)):
        text = str(adviceResultSoSo.Classifier[ssr]) + ' with ' + str(adviceResultSoSo.Algorithm[ssr])
        listOfPrettyGoodResult.append(Button(description=text,layout=Layout(width="auto"),disabled=True,icon='star-o', button_style='warning'))
    for br in range(len(adviceResultBAD)):
        text = str(adviceResultBAD.Classifier[br]) + ' with ' + str(adviceResultBAD.Algorithm[br])
        listOfPrettyGoodResult.append(Button(description=text,layout=Layout(width="auto"),disabled=True,icon='close', button_style='danger'))
        
    out14 = widgets.Output()
    out15 = widgets.Output()
    trainingResult.children = [out13,out14,out15]
    trainingResult.set_title(1,"Result:")
    trainingResult.set_title(2,"Advice which classifier you should use:")
    
    if len(listOfTrainedRunningClassifiers)!=0:
        predictionButton.disabled=False
    else:
        predictionButton.disabled=True
    
    activatePredictionButton()
    
    with out14:
        display(adviceResult)
    with out15:
        display(HBox(listOfPrettyGoodResult,layout=Layout(width='100%',display='inline-flex',flex_flow='row wrap')))
    
    runModelButton.icon = "gear"

In [45]:
output3 = widgets.Output()

layout = Layout(display='flex',
                align_items='center',
                width='50%')

runModelButton = Button(description="Run Model", tooltip="Learn Model on different Algorithms to give you an advice",
                       layout = Layout(width="auto"), icon="gear", disabled=True)
runModelButton.on_click(runModelClickEvent)

myModelAlgorithms = Label(value='▶ Choose Classifiers you want to train model on : ', layout= Layout(width="auto"))
mymodelLabel = Label(value='▶ Now run learning model : ', layout= Layout(width="auto"))

BRNBButton = Button(description="Binary Relevance with NomialNB", disabled=False, layout = layout, icon="plus-square")
BRRFButton = Button(description="Binary Relevance with RandomForestClassifier", disabled=False, layout = layout, icon="plus-square")

CCNBButton = Button(description="ClassifierChain with NomialNB", disabled=False, layout = layout, icon="plus-square")
CCRFButton = Button(description="Classifier Chain with RandomForestClassifier", disabled=False, layout = layout, icon="plus-square")

LPNBButton = Button(description="LabelPowerset with NomialNB", disabled=False, layout = layout, icon="plus-square")
LPRFButton = Button(description="LabelPowerset with RandomForestClassifier", disabled=False, layout = layout, icon="plus-square")

MLkNNButton = Button(description="MLkNN", disabled=False, layout = layout, icon="plus-square")
BRMLkNNButton = Button(description="BRMLkNN", disabled=False, layout = layout, icon="plus-square")

k = IntSlider(description= "k =", min=2, max=10, step=1, readout=True, value=3, disabled = True)

listOfButtons = [BRNBButton, BRRFButton, CCNBButton, CCRFButton, LPNBButton, LPRFButton, MLkNNButton, BRMLkNNButton]

global choosenClassifier
choosenClassifier = []

global allSelected

BRNBButton.on_click(check_event)
BRRFButton.on_click(check_event)
CCNBButton.on_click(check_event)
CCRFButton.on_click(check_event)
LPNBButton.on_click(check_event)
LPRFButton.on_click(check_event)
MLkNNButton.on_click(check_event)
BRMLkNNButton.on_click(check_event)

BR = HBox([BRNBButton, BRRFButton])
CC = HBox([CCNBButton, CCRFButton])
LP = HBox([LPNBButton, LPRFButton])
KN = HBox([MLkNNButton, BRMLkNNButton])

selectAllButton = Button(description="Select All Unselected", layout=Layout(width="auto"), icon='check-square')
selectAllButton.on_click(selectAllButtonOnClick)
UnselectAllButton = Button(description="UnSelect All Selected", layout=Layout(width="auto"), icon='square-o')
UnselectAllButton.on_click(selectAllButtonOnClick)


display(HBox([myModelAlgorithms,selectAllButton,UnselectAllButton,k]))
display(VBox([BR,CC,LP,KN]))
display(HBox([mymodelLabel,runModelButton]))

display(output3)

Output()

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning:

Pass n_neighbors=3 as keyword args. From version 0.25 passing these as positional arguments will result in an error

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning:

Pass n_neighbors=3 as keyword args. From version 0.25 passing these as positional arguments will result in an error



***
## <img style="float: left" src="./icons/prediction.png" width="35px" height="35px"/>&nbsp; Predict volunteer competency:

In [46]:
# Functions for prediction

# Getting the selected value from dropListClassifier
def changeEvent(change):
    global selection
    allOptions = dropListClassifier.options
    allKeys = list(allOptions.keys())
    allValues = list(allOptions.values())
    for i in allValues:
        if dropListClassifier.value == i:
            selection = i
    return selection

# -----------------------------------------------------------------------------------------------------------------------------
# Prediction Button Click
def predictionClickEvent(event):
    # clearing output4
    output4.clear_output()
    
    # Get pre-processed dataset
    generatedDataset = getFinalPreprocessedDataset()
    
    # taking the same columns of training dataset
    columnsWithoutOutput = generatedDataset[["NumberOfAccomplishedTasks", "OrganizationManagementProportion", "TrainingLearningProportion", "CoorporationProportion", "VarietyWorkProportion", "StartWorkingMorning", "WorkingLongTime", "taskLocation"]].values
    predictionResult = np.array(columnsWithoutOutput).reshape((1, -1))
    
    # Prediction process using the choosed classifier
    predictionClassifier = listOfTrainedRunningClassifiers[selection]
    predictionClassifier = predictionClassifier.predict(predictionResult).toarray()
    
    # Making result in a pretty output    
    for item in predictionClassifier:
        finalPredictionResult = item
    
    volumnteerName = generatedDataset.personName[0]
    
    predictionLabel = Label(value="Prediction "+str(volumnteerName)+"'s competencies using "+str(listOfTrainedRunningClassifiers[selection]),
                           layout = Layout(width='auto'))
    
    listOfPredictedResult = [];
    for counter in range(len(finalPredictionResult)):
        if counter==0: # Self Management Competency
            if finalPredictionResult[counter] == 0:
                listOfPredictedResult.append(Button(description="Self Management",layout=Layout(width="auto"),disabled=True,icon='close', button_style='danger'))
            else:
                listOfPredictedResult.append(Button(description="Self Management",layout=Layout(width="auto"),disabled=True,icon='check', button_style='success'))
        elif counter==1: # Willingness To Learn Competency
            if finalPredictionResult[counter] == 0:
                listOfPredictedResult.append(Button(description="Willingness To Learn",layout=Layout(width="auto"),disabled=True,icon='close', button_style='danger'))
            else:
                listOfPredictedResult.append(Button(description="Willingness To Learn",layout=Layout(width="auto"),disabled=True,icon='check', button_style='success'))
        elif counter==2: # Energy Competency
            if finalPredictionResult[counter] == 0:
                listOfPredictedResult.append(Button(description="Energy",layout=Layout(width="auto"),disabled=True,icon='close', button_style='danger'))
            else:
                listOfPredictedResult.append(Button(description="Energy",layout=Layout(width="auto"),disabled=True,icon='check', button_style='success'))
        elif counter==3: # Persistence Competency
            if finalPredictionResult[counter] == 0:
                listOfPredictedResult.append(Button(description="Persistence",layout=Layout(width="auto"),disabled=True,icon='close', button_style='danger'))
            else:
                listOfPredictedResult.append(Button(description="Persistence",layout=Layout(width="auto"),disabled=True,icon='check', button_style='success'))
        elif counter==4: # Mobility Competency
            if finalPredictionResult[counter] == 0:
                listOfPredictedResult.append(Button(description="Mobility",layout=Layout(width="auto"),disabled=True,icon='close', button_style='danger'))
            else:
                listOfPredictedResult.append(Button(description="Mobility",layout=Layout(width="auto"),disabled=True,icon='check', button_style='success'))
    
    out16 = widgets.Output();
    out17 = widgets.Output();
    with output4:
        display(VBox([out16,out17]))
        with out16:
            display(predictionLabel)
        with out17:
            display(HBox(listOfPredictedResult,layout=Layout(width='100%',display='inline-flex',flex_flow='row wrap')))

In [1]:
output4 = widgets.Output()
layout = Layout(width="auto")

dropListClassifier = Dropdown(
            options={'':0},
            value=0,
            description='Choose Algorithm/Estimator:',
            layout = layout,
            style= {'description_width':'auto', 'description_color':'red'}
            )
dropListClassifier.observe(changeEvent, names=['value'])

predictionButton = Button(description="Predict volunteer competencies", layout =Layout(width="auto"), icon='question-circle', disabled=True)
predictionButton.on_click(predictionClickEvent)

items = HBox([dropListClassifier])

display(items)

display(predictionButton)
display(output4)

NameError: name 'widgets' is not defined

In [2]:
!pip freeze > requirements.txt